## 1 SpeechBrain

## 1.1 测试

In [1]:
import torchaudio
import torch

In [2]:
from speechbrain.pretrained import SpeakerRecognition
verification = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir="pretrained_models/spkrec-ecapa-voxceleb")

torchvision is not available - cannot save figures


#### 同一人不同句子

In [3]:
score1, prediction1 = verification.verify_files("tests/samples/my_test/1.aac", "tests/samples/my_test/2.aac") 
print(score1,prediction1)

tensor([0.5786]) tensor([True])


In [4]:
audio1 = verification.load_audio("tests/samples/my_test/1.aac")
audio2 = verification.load_audio("tests/samples/my_test/2.aac")
score2, prediction2 = verification.verify_batch(audio1, audio2,threshold=0.3) # 同一人不同句子
print(score2[0],prediction2[0])

tensor([0.5786]) tensor([True])


#### 不同一人同句子

In [5]:
audio3 = verification.load_audio("tests/samples/my_test/3.m4a")
score3, prediction3 = verification.verify_batch(audio1, audio3,threshold=0.3) 
print(score3[0],prediction3[0])

tensor([0.2450]) tensor([False])


#### 同一人同句子

In [6]:
audio4 = verification.load_audio("tests/samples/my_test/4.wav")
score4, prediction4 = verification.verify_batch(audio2, audio4,threshold=0.3) 
print(score4[0],prediction4[0])

tensor([0.4977]) tensor([True])


## 1.2 SpeechBrain模块代码

### 1.2.1 提取特征

In [7]:
audio1 = verification.load_audio("tests/samples/my_test/1.aac")
embeddings1 = verification.encode_batch(audio1)
embeddings1.shape

torch.Size([1, 1, 192])

In [8]:
audio2 = verification.load_audio("tests/samples/my_test/2.aac")
embeddings2 = verification.encode_batch(audio2)
embeddings2.shape

torch.Size([1, 1, 192])

In [9]:
audio3 = verification.load_audio("tests/samples/my_test/3.m4a")
embeddings3 = verification.encode_batch(audio3)
embeddings3.shape

torch.Size([1, 1, 192])

In [10]:
audio4 = verification.load_audio("tests/samples/my_test/4.wav")
embeddings4 = verification.encode_batch(audio4)
embeddings4.shape

torch.Size([1, 1, 192])

### 1.2.2 计算相似度

In [11]:
similarity = torch.nn.CosineSimilarity(dim=-1, eps=1e-6)
score = similarity(embeddings1, embeddings2)
print(f"Eembeddings Score: {score}")

Eembeddings Score: tensor([[0.5786]])


In [12]:
score = similarity(embeddings2, embeddings3)
print(f"Eembeddings Score: {score}")

Eembeddings Score: tensor([[0.2261]])


In [13]:
score = similarity(embeddings2, embeddings4)
print(f"Eembeddings Score: {score}")

Eembeddings Score: tensor([[0.4977]])


In [14]:
score = similarity(embeddings1, embeddings4)
print(f"Eembeddings Score: {score}")

Eembeddings Score: tensor([[0.5315]])


# 2 百度PaddleSpeech

## 2.1 测试

#### 同一人不同句子

In [15]:
import paddle
from paddlespeech.cli.vector import VectorExecutor

vector_executor = VectorExecutor()
audio_emb = vector_executor(
    model='ecapatdnn_voxceleb12',
    sample_rate=16000,
    config=None,  # Set `config` and `ckpt_path` to None to use pretrained model.
    ckpt_path=None,
    audio_file='tests/samples/ASR/spk1_snt1.wav',
    device=paddle.get_device())
# print('Audio embedding Result: \n{}'.format(audio_emb))

test_emb = vector_executor(
    model='ecapatdnn_voxceleb12',
    sample_rate=16000,
    config=None,  # Set `config` and `ckpt_path` to None to use pretrained model.
    ckpt_path=None,
    audio_file='tests/samples/ASR/spk1_snt6.wav',
    device=paddle.get_device())
# print('Test embedding Result: \n{}'.format(test_emb))

# score range [0, 1]
score = vector_executor.get_embeddings_score(audio_emb, test_emb)
print(f"Eembeddings Score: {score}")

Eembeddings Score: 0.6716901659965515


#### 不同人不同句子

In [16]:
audio_emb = vector_executor(
    model='ecapatdnn_voxceleb12',
    sample_rate=16000,
    config=None,  # Set `config` and `ckpt_path` to None to use pretrained model.
    ckpt_path=None,
    audio_file='tests/samples/ASR/spk1_snt1.wav',
    device=paddle.get_device())
print(audio_emb.shape)

test_emb = vector_executor(
    model='ecapatdnn_voxceleb12',
    sample_rate=16000,
    config=None,  # Set `config` and `ckpt_path` to None to use pretrained model.
    ckpt_path=None,
    audio_file='tests/samples/ASR/spk2_snt6.wav',
    device=paddle.get_device())
print(test_emb.shape)

# score range [0, 1]
score = vector_executor.get_embeddings_score(audio_emb, test_emb)
print(f"Eembeddings Score: {score}")

(192,)
(192,)
Eembeddings Score: 0.08822208642959595
